In [1]:
import numpy as np
import math

In [2]:
from ase.io import read
atoms=read('data.vasp',format='vasp')

#all atom is in central region by definition
#here we assume number of ghost shell is 1

#construct the list of vector connecting between each atoms

masses=atoms.get_masses()
natom=len(atoms.positions)

dist=np.zeros((natom,natom,3))
for i in range(natom):
    for j in range(natom):
        rmin=np.zeros(3)
        rij=atoms.get_distance(i,j,vector=True)
        rij_mic=atoms.get_distance(i,j,mic=True,vector=True)
        if(np.linalg.norm(rij_mic)<np.linalg.norm(rij)):
            rmin=rij_mic
        else:
            rmin=rij
        dist[i,j]=rmin

In [3]:
with open('FORCE_CONSTANTS_2ND','r') as fc:
    lines=fc.readlines()

    nlineblock=4
    fc_all=np.zeros((natom,natom,3,3))
    start=1
    for i in range(natom):
        for j in range(natom):
            fc_block=lines[start+1:start+nlineblock]
            fc=np.loadtxt(fc_block)
            fc_all[i,j]=fc/np.sqrt(masses[i]*masses[j])
            #fc_all[i,j]=fc
            start=start+nlineblock



In [4]:
fc_all[0,1]

array([[ 0.03913539, -0.        , -0.        ],
       [ 0.        , -0.00978188, -0.        ],
       [ 0.        , -0.        , -0.00978188]])

In [5]:
atoms.get_masses()

array([28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085])

In [6]:
dist

array([[[ 0.    ,  0.    ,  0.    ],
        [ 0.    , -2.725 , -2.725 ],
        [-2.725 ,  0.    , -2.725 ],
        [-2.725 , -2.725 ,  0.    ],
        [ 1.3625,  1.3625,  1.3625],
        [ 1.3625, -1.3625, -1.3625],
        [-1.3625,  1.3625, -1.3625],
        [-1.3625, -1.3625,  1.3625]],

       [[ 0.    ,  2.725 ,  2.725 ],
        [ 0.    ,  0.    ,  0.    ],
        [-2.725 ,  2.725 ,  0.    ],
        [-2.725 ,  0.    ,  2.725 ],
        [ 1.3625, -1.3625, -1.3625],
        [ 1.3625,  1.3625,  1.3625],
        [-1.3625, -1.3625,  1.3625],
        [-1.3625,  1.3625, -1.3625]],

       [[ 2.725 ,  0.    ,  2.725 ],
        [ 2.725 , -2.725 ,  0.    ],
        [ 0.    ,  0.    ,  0.    ],
        [ 0.    , -2.725 ,  2.725 ],
        [-1.3625,  1.3625, -1.3625],
        [-1.3625, -1.3625,  1.3625],
        [ 1.3625,  1.3625,  1.3625],
        [ 1.3625, -1.3625, -1.3625]],

       [[ 2.725 ,  2.725 ,  0.    ],
        [ 2.725 ,  0.    , -2.725 ],
        [ 0.    ,  2.725 , -2.72

In [7]:
#slice for x-component
dist[:,:,0]

array([[ 0.    ,  0.    , -2.725 , -2.725 ,  1.3625,  1.3625, -1.3625,
        -1.3625],
       [ 0.    ,  0.    , -2.725 , -2.725 ,  1.3625,  1.3625, -1.3625,
        -1.3625],
       [ 2.725 ,  2.725 ,  0.    ,  0.    , -1.3625, -1.3625,  1.3625,
         1.3625],
       [ 2.725 ,  2.725 ,  0.    ,  0.    , -1.3625, -1.3625,  1.3625,
         1.3625],
       [-1.3625, -1.3625,  1.3625,  1.3625,  0.    ,  0.    ,  2.725 ,
         2.725 ],
       [-1.3625, -1.3625,  1.3625,  1.3625,  0.    ,  0.    ,  2.725 ,
         2.725 ],
       [ 1.3625,  1.3625, -1.3625, -1.3625, -2.725 , -2.725 ,  0.    ,
         0.    ],
       [ 1.3625,  1.3625, -1.3625, -1.3625, -2.725 , -2.725 ,  0.    ,
         0.    ]])

In [8]:
Rx=dist[:,:,0]
Ry=dist[:,:,1]
Rz=dist[:,:,2]
Rx.shape, fc_all.shape

((8, 8), (8, 8, 3, 3))

コードをよく見ると、ix,jxの要素では負が入ってた。
当初比較していたのはFortran orderでのダンプだったので、列と行の順番が逆。

In [9]:
Vx=np.zeros((natom,natom,3,3))
Vy=np.zeros((natom,natom,3,3))
Vz=np.zeros((natom,natom,3,3))

for i in range(natom):
    for j in range(natom):
        Vijx=-Rx[i,j]*fc_all[i,j]
        Vx[i,j]=Vijx

        Vijy=-Ry[i,j]*fc_all[i,j]
        Vy[i,j]=Vijy

        Vijz=-Rz[i,j]*fc_all[i,j]
        Vz[i,j]=Vijz



In [10]:
#compare with GULP output
#1st atom x-direction component
Vx[0,:,0,:]

array([[-0.        ,  0.        ,  0.        ],
       [-0.        ,  0.        ,  0.        ],
       [-0.02665562,  0.        , -0.        ],
       [-0.02665562, -0.        ,  0.        ],
       [ 0.21791282,  0.13620392,  0.13620392],
       [ 0.21791282, -0.13620392, -0.13620392],
       [-0.21791282,  0.13620392, -0.13620392],
       [-0.21791282, -0.13620392,  0.13620392]])

GULP side
```
 ixyz:           1
  0.000000000000000E+000  0.000000000000000E+000  0.000000000000000E+000
  0.000000000000000E+000  0.000000000000000E+000  0.000000000000000E+000
 -2.665087135698760E-002  8.616184384773376E-017 -7.000649812628367E-017
 -2.665087135698760E-002 -7.000649812628367E-017  8.616184384773376E-017
  0.217874033757599       0.136179679902043       0.136179679902043     
  0.217874033757600      -0.136179679902044      -0.136179679902044     
 -0.217874033757599       0.136179679902043      -0.136179679902043     
 -0.217874033757599      -0.136179679902043       0.136179679902043    
```

In [11]:
#1st atom y-direction component
Vx[0,:,1,:]

array([[ 0.        , -0.        ,  0.        ],
       [-0.        ,  0.        ,  0.        ],
       [-0.        ,  0.10664395, -0.        ],
       [-0.        , -0.02665562,  0.        ],
       [ 0.13620392,  0.21791282,  0.13620392],
       [-0.13620392,  0.21791282,  0.13620392],
       [ 0.13620392, -0.21791282,  0.13620392],
       [-0.13620392, -0.21791282,  0.13620392]])

GULP
```
  0.000000000000000E+000  0.000000000000000E+000  0.000000000000000E+000
  0.000000000000000E+000  0.000000000000000E+000  0.000000000000000E+000
 -8.616184384773376E-017  0.106624964870440      -8.616184384773376E-017
 -7.000649812628367E-017 -2.665087135698760E-002  8.616184384773376E-017
  0.136179679902043       0.217874033757599       0.136179679902043     
 -0.136179679902044       0.217874033757600       0.136179679902043     
  0.136179679902043      -0.217874033757599       0.136179679902043     
 -0.136179679902043      -0.217874033757599       0.136179679902043   
```

In [12]:
Vx[7,:,2,:]

array([[-0.13620392, -0.13620392,  0.21791282],
       [ 0.13620392, -0.13620392,  0.21791282],
       [ 0.13620392, -0.13620392, -0.21791282],
       [-0.13620392, -0.13620392, -0.21791282],
       [ 0.        , -0.        ,  0.10664395],
       [-0.        , -0.        , -0.02665562],
       [ 0.        ,  0.        ,  0.        ],
       [-0.        ,  0.        , -0.        ]])

GULP
```
-0.136179679902043      -0.136179679902043       0.217874033757599     
  0.136179679902043      -0.136179679902043       0.217874033757599     
  0.136179679902043      -0.136179679902043      -0.217874033757599     
 -0.136179679902043      -0.136179679902043      -0.217874033757599     
  0.000000000000000E+000  5.385115240483360E-018 -0.106624964870440     
  3.231069144290016E-017  1.615534572145008E-017  2.665087135698754E-002
  0.000000000000000E+000  0.000000000000000E+000  0.000000000000000E+000
  0.000000000000000E+000  0.000000000000000E+000  0.000000000000000E+000
```

In [13]:
#y-component

Vy[0,:,0,:]

array([[-0.        ,  0.        ,  0.        ],
       [ 0.10664395, -0.        , -0.        ],
       [ 0.        , -0.        ,  0.        ],
       [-0.02665562, -0.        ,  0.        ],
       [ 0.21791282,  0.13620392,  0.13620392],
       [-0.21791282,  0.13620392,  0.13620392],
       [ 0.21791282, -0.13620392,  0.13620392],
       [-0.21791282, -0.13620392,  0.13620392]])

GULP
```
  0.000000000000000E+000  0.000000000000000E+000  0.000000000000000E+000
  0.106624964870440      -8.616184384773376E-017 -8.616184384773376E-017
  0.000000000000000E+000  0.000000000000000E+000  0.000000000000000E+000
 -2.665087135698760E-002 -7.000649812628367E-017  8.616184384773376E-017
  0.217874033757599       0.136179679902043       0.136179679902043     
 -0.217874033757599       0.136179679902043       0.136179679902043     
  0.217874033757600      -0.136179679902044       0.136179679902043     
 -0.217874033757599      -0.136179679902043       0.136179679902043   
```

In [14]:
#z component
Vz[0,:,0,:]

array([[-0.        ,  0.        ,  0.        ],
       [ 0.10664395, -0.        , -0.        ],
       [-0.02665562,  0.        , -0.        ],
       [ 0.        ,  0.        , -0.        ],
       [ 0.21791282,  0.13620392,  0.13620392],
       [-0.21791282,  0.13620392,  0.13620392],
       [-0.21791282,  0.13620392, -0.13620392],
       [ 0.21791282,  0.13620392, -0.13620392]])

GULP
```
  0.000000000000000E+000  0.000000000000000E+000  0.000000000000000E+000
  0.106624964870440      -8.616184384773376E-017 -8.616184384773376E-017
 -2.665087135698760E-002  8.616184384773376E-017 -7.000649812628367E-017
  0.000000000000000E+000  0.000000000000000E+000  0.000000000000000E+000
  0.217874033757599       0.136179679902043       0.136179679902043     
 -0.217874033757599       0.136179679902043       0.136179679902043     
 -0.217874033757599       0.136179679902043      -0.136179679902043     
  0.217874033757600       0.136179679902043      -0.136179679902044     
```

convert the shape of tensor to evaluate Sij with dot product with eigenvector

In [15]:
flatVx=np.reshape(Vx.transpose(0,2,1,3),(24,24))

In [16]:
flatVx[0]

array([-0.        ,  0.        ,  0.        , -0.        ,  0.        ,
        0.        , -0.02665562,  0.        , -0.        , -0.02665562,
       -0.        ,  0.        ,  0.21791282,  0.13620392,  0.13620392,
        0.21791282, -0.13620392, -0.13620392, -0.21791282,  0.13620392,
       -0.13620392, -0.21791282, -0.13620392,  0.13620392])

In [17]:
GULP_Vijx=np.loadtxt('Vijx.dat').reshape((24,24))

In [18]:
GULP_Vijx[0]

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -2.66508714e-02,  8.61618438e-17,
       -7.00064981e-17, -2.66508714e-02, -7.00064981e-17,  8.61618438e-17,
        2.17874034e-01,  1.36179680e-01,  1.36179680e-01,  2.17874034e-01,
       -1.36179680e-01, -1.36179680e-01, -2.17874034e-01,  1.36179680e-01,
       -1.36179680e-01, -2.17874034e-01, -1.36179680e-01,  1.36179680e-01])

In [19]:
np.sum(flatVx-GULP_Vijx)

0.0

In [20]:
flatVy=np.reshape(Vy.transpose(0,2,1,3),(24,24))
GULP_Vijy=np.loadtxt('Vijy.dat').reshape((24,24))
np.sum(flatVy-GULP_Vijy)

4.163336342344337e-17

In [21]:
flatVz=np.reshape(Vz.transpose(0,2,1,3),(24,24))
GULP_Vijz=np.loadtxt('Vijz.dat').reshape((24,24))
np.sum(flatVz-GULP_Vijz)

-2.7755575615628914e-17